In [1]:
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
import torch

# Initialize tokenizer for corpus of bert-large-uncased
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad', resume_download=True)

# Initialize model BertForQuestionAnswering for bert-large-uncased
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad', resume_download=True)

def answer_question(question, answer_text):
    '''
    Lấy input là chuỗi string của câu question và answer_text chứa nội dung câu trả lời của câu question.
    Xác định từ trong answer_text là câu trả lời và in ra.
    '''
    # ======== Tokenize ========
    # Áp dụng tokenizer cho cặp câu <question, answer_text>. input_ids là concatenate indice của cả 2 câu sau khi đã thêm các token CLS và SEP như mô tả trong tác vụ Question and Answering.
    input_ids = tokenizer.encode(question, answer_text)

    # ======== Set Segment IDs ========
    # Xác định vị trí đầu tiên chứa token [SEP] trong câu.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    sep_index = tokens.index('[SEP]')

    # Tạo segment index đánh dấu các vị trí từ thuộc question (giá trị 0) và answer_text (giá trị 1)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    print(segment_ids)

    # Kiểm tra độ dài segment_ids phải bằng input_ids
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Dự báo phân phối xác suất của vị trí của từ start và từ end trong chuỗi concatenate <question, answer_text> mà chứa kết quả cho câu trả lời.
    out = model(torch.tensor([input_ids]), # chuỗi index biểu thị cho inputs.
                                    token_type_ids=torch.tensor([segment_ids])) # chuỗi index thành phần segment câu để phân biệt giữa câu question và câu answer_text

    # ======== Reconstruct Answer ========
    # Tìm ra vị trí start, end với score là cao nhất
    answer_start = torch.argmax(out['start_logits'])
    answer_end = torch.argmax(out['end_logits'])

    # Chuyển ngược từ input_ids sang list tokens
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Token đầu tiên của câu trả lời
    answer = tokens[answer_start]

    # Lựa chọn các thành phần còn lại của câu trả lời và join chúng với whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # Nếu token là một subword token (có dấu ## ở đầu) thì combine vào answer bằng token gốc (loại bỏ dấu ##).
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        # Nếu trái lại thì combine trực tiếp vào answer.
        else:
            answer += ' ' + tokens[i]
    return answer

/home/tuannm/anaconda3/envs/chatbotdocs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
from googletrans import Translator

def vi_to_en(text):
  translator = Translator()
  translation = translator.translate(text, src='vi', dest='en')
  return translation.text

def en_to_vi(text):
  translator = Translator()
  translation = translator.translate(text, src='en', dest='vi')
  return translation.text

In [5]:
question = vi_to_en("""bố của thạch thanh lâm kinh doanh gì""")
paragraph = vi_to_en("Xuyên qua sảnh lớn, có một phòng nghỉ dành riêng cho khách VIP. Thạch Thanh Lâm tiến vào, bên trong chỉ có một người đàn ông trung niên, trên người mặc bộ đồ âu phục vừa vặn, ông ấy không ngồi mà đứng cạnh sofa, nhìn thấy anh thì lập tức tiến lên một bước, “Cậu chủ, lâu lắm rồi không gặp cậu.” “Đúng là lâu lắm rồi không gặp, chú Trần.”, thấy ông ấy đứng, Thạch Thanh Lâm liền ngồi luôn xuống tay vịn sofa. Chú Trần là thư ký của bố anh, theo bố anh đã mấy chục năm, vẫn luôn gọi bố anh là ông chủ, gọi anh là cậu chủ. Từ khi còn trẻ đến giờ, ngoài đôi nét đã già đi, thì hầu như chẳng có gì thay đổi. “Tính ra cũng phải hơn chục năm rồi, từ khi cậu ra nước ngoài là không còn gặp cậu nữa.”, chú Trần cảm thán, “Có phải cậu nên về thăm bố cậu một lần không?” Thạch Thanh Lâm xoay xoay dây đồng hồ trên cổ tay, chỉ vài giây, trong đầu anh đã cân nhắc xong rồi, “Chắc là không cần đâu, vừa rồi lúc ông ta đi vào hội trường cùng ban tổ chức, tôi nhìn thấy ông ta rồi.” Lúc ấy, khi nhóm người tiến vào, anh vừa liếc mắt đã nhìn thấy, bố anh cũng đưa mắt nhìn về phía anh, mặc dù không biết có phải là nhìn anh hay không. Anh cũng không ngờ lại gặp ông ta trong trường hợp này, bố anh kinh doanh trong rất nhiều lĩnh vực, chỉ riêng ngành game là chưa từng động vào. Chú Trần nói: “Ý tôi là về nhà gặp, đã bao nhiêu năm rồi hai bố con cậu không cùng ngồi xuống ăn một bữa cơm chung với nhau.")

en_to_vi(answer_question(question, paragraph))



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

'.'

## PyPDF2

In [50]:
import PyPDF2
import re


def extract_text_from_pdf(pdf_file: str) -> [str]:
    reader = PyPDF2.PdfReader(pdf_file)
    pdf_text = []
    
    
    for page in reader.pages:
        content = page.extract_text()
        pdf_text.append(content)
        
    return pdf_text
    

In [53]:
content = extract_text_from_pdf("../Lâm Nam - Thiên Như Ngọc.pdf")

In [55]:
content[273]

'Biết chắc tên kia đã đi rồi, Thạch Thanh Lâm mới quay đầu đi vào nhà vệ sinh.\n***\nĐồ Nam chống hai tay lên mép bồn rửa mặt, nhìn vào gương mà khẽ nhíu mày .\nHành động này thật ra có chút không ổn, nếu người đến là một người xa lạ thì\ncòn đ ược, có điều  lại là T iết Thành, bị anh ta chứng kiến cô ở đây th ì khó mà\ngiải thích rõ ràng được.\nCó tiếng gõ cửa, Thạch Thanh Lâm đẩy cửa vào.\nAnh không tiến vào trong, mà chỉ đứng ở cửa nhìn cô.\nNhà vệ sinh không hề nhỏ, thậm chí là khá rộng, nhưng anh đứng ở cửa khiến\nĐồ Nam bỗng cảm thấy rất chật chội, dường như còn chẳng có lối ra, chỉ chừa\nlại đúng một chút không gian giữa hai người họ.\n“Đi rồi à?”, cô chỉ hỏi mỗi vậy .\n“Ừm.”, T hạch Thanh Lâm nói: “Cậ u ta chỉ lên nói mấy câu vớ vẩn thôi, không\nphát hiện ra gì cả.”\nKhông nói câu này còn đỡ, nói ra rồi mới lại xấu hổ. Đồ Nam im lặng, cụp mắt\nxuống, đột nhiên l ại chú ý đến một chi tiết, cô vẫn đi giày của mình, cò n Thạch\nThanh Lâm thì đã thay dép lê. Dù anh sống một mình nh

In [61]:
question = vi_to_en("""Đồ nam làm gì""")

print(question)
paragraph = vi_to_en(content[273])

en_to_vi(answer_question(question, paragraph))

What do men do
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

'[CLS] Đàn ông làm gì [sep] khi biết rằng anh chàng kia đã biến mất, Thach Thanh Lam quay sang nhà vệ sinh.* * * tay của đàn ông trên mép bồn rửa, nhìn vào gương và nhíu mày'

## pdfplumber

In [4]:
import pdfplumber

def plumber_read(pdffile):
    content = []
    with pdfplumber.open(pdffile) as pdf:
        for page in pdf.pages:
            content.append(page.extract_text())
            
    return content


text = plumber_read("../Hoàng Minh Sơn- BB nghiệm thu CTV-T9.2023.pdf")


text[4]

'II. CHI PHÍ THANH TOÁN\nKhối lượng công việc đã hoàn thành ……………..%\nHai bên xác nhận thù lao chi trả cho Cộng tác viên để thực hiện công việc là:\n…..................…….......................................................................................................... VNĐ\n(Bằng chữ: .........................................................................................................................../.).\nSau khi khấu trừ thuế thu nhập cá nhân tại nguồn, số tiền Cộng tác viên được thanh toán\nlà: …..................………………………………………………………………………. VNĐ\n(Bằng chữ: .............................................................................../.).\nIII. KẾT LUẬN:\n1. Cộng tác viên đã hoàn thành khối lượng công việc đúng theo hợp đồng đã ký.\n2. Các công việc được Người Quản lý và theo dõi công việc của ISOFH xác nhận đủ điều\nkiện thanh toán.\n3. Hai bên thống nhất với nội dung Biên bản nghiệm thu và lấy làm cơ sở thanh toán chi phí\ncho Cộng tác viên.\n4. Biên bản này được lập 